In [26]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import math
import yfinance as yf
import requests as rq
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [28]:
aapl = yf.Ticker("AAPL")

# get stock info
aapl.info

# get historical market data
hist = aapl.history(period="48mo")
hist

Open        High         Low       Close    Volume  \
Date                                                                   
2018-07-09   45.545465   45.829071   45.497397   45.805038  79026400   
2018-07-10   45.836288   45.973283   45.708902   45.749763  63756400   
2018-07-11   45.305122   45.612764   45.091215   45.156109  75326000   
2018-07-12   45.552675   46.004525   45.499798   45.913193  72164400   
2018-07-13   45.925208   46.107869   45.881944   45.985294  50055600   
...                ...         ...         ...         ...       ...   
2022-07-01  136.039993  139.039993  135.660004  138.929993  71007500   
2022-07-05  137.770004  141.610001  136.929993  141.559998  73353800   
2022-07-06  141.350006  144.119995  141.080002  142.919998  74064300   
2022-07-07  143.289993  146.550003  143.279999  146.350006  66253700   
2022-07-08  145.259995  147.550003  145.000000  147.039993  64493200   

            Dividends  Stock Splits  
Date                                 
2018-07-09        0.0           0.0  
2018-07-10        0.0           0.0  
2018-07-11        0.0           0.0  
2018-07-12        0.0           0.0  
2018-07-13        0.0           0.0  
...               ...           ...  
2022-07-01        0.0           0.0  
2022-07-05        0.0           0.0  
2022-07-06        0.0           0.0  
2022-07-07        0.0           0.0  
2022-07-08        0.0           0.0  

[1008 rows x 7 columns]

In [29]:
#Get the shape of the rows and columns in the dataset
hist.shape

(1008, 7)

In [30]:
hist.dtypes

Open            float64
High            float64
Low             float64
Close           float64
Volume            int64
Dividends       float64
Stock Splits    float64
dtype: object

In [31]:
#Visulaize
plt.figure(figsize=(16,8))
plt.title('Apple 48 Months Closing Price History')
plt.plot(hist['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.show()

In [32]:
#Create a new dataframe with only the 'Close' column
data =  hist.filter(['Close'])
#Convert the dataframe to a numpy array
dataset = data.values
#Get the number of rows to train the model on
training_dataset_len = math.ceil(len(dataset) * .8)

training_dataset_len

807

In [33]:
#Scale the data (Good practice prior to passing to neuro network)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

array([[0.07743712],
       [0.07706135],
       [0.07302556],
       ...,
       [0.73764535],
       [0.76096327],
       [0.76565395]])

In [34]:
scaled_data.shape

(1008, 1)

In [35]:
#Create the training data set
#Create the scaled training data set
# Creating a data structure with 60 timesteps and 1 output
train_data = scaled_data[0:training_dataset_len, :]
#Split the data
x_train = []
y_train = []

for i in range(60, len(train_data)):
  x_train.append(train_data[i-60:i, 0])
  y_train.append(train_data[i, 0])
  if i<= 60:
    print(x_train)
    print(y_train)
    print()

[array([0.07743712, 0.07706135, 0.07302556, 0.07817238, 0.07866254,
       0.07797632, 0.07885859, 0.07714299, 0.0795612 , 0.07884231,
       0.07912005, 0.08139122, 0.08436497, 0.08336826, 0.07809069,
       0.07634238, 0.07696327, 0.09527953, 0.10490331, 0.10588366,
       0.1076483 , 0.10444582, 0.10467458, 0.10733786, 0.10632125,
       0.10851844, 0.10996131, 0.11076471, 0.11581483, 0.12279978,
       0.11932373, 0.11863505, 0.11865147, 0.11937288, 0.12047153,
       0.12339005, 0.12627586, 0.1316539 , 0.1350152 , 0.13927826,
       0.14047518, 0.13803218, 0.13185063, 0.12889928, 0.12402948,
       0.13308043, 0.12852219, 0.13727786, 0.13306399, 0.12329166,
       0.12388195, 0.12409507, 0.1268169 , 0.12293093, 0.12806307,
       0.13035851, 0.12745636, 0.13488397, 0.13617936, 0.13867158])]
[0.14198365958891998]



In [36]:
#Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [37]:
#Reshape the data
#No of Samples, No of time steps (columns) and the no. of features (closing price)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(747, 60, 1)

In [38]:
#Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1))) #50 neurons
model.add(LSTM(50, return_sequences=False ))
model.add(Dense(25))
model.add(Dense(1))

In [39]:
#Compile the model
model.compile(optimizer='adam', loss='mean_squared_error') #optimzer is used to improve upon loss fucntion

In [40]:
#Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

747/747 [==============================] - 41s 46ms/step - loss: 0.0023


In [41]:
#Creating the testing dataset
#Create a new array containing scaled values from 1552 to 2005
test_data = scaled_data[training_dataset_len - 60: , :] #scaled data minus 60 to the end of the dataset and all columns
#Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_dataset_len:, :] #the values for the models to predict, the 61st values that is unscaled
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0]) #x_test contains the past 60 values

In [42]:
#Convert  the test data to a numpy array for use in the LSTM model
x_test = np.array(x_test)

In [43]:
x_test.shape

(201, 60)

In [44]:
#Reshape the test data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1)) #no of rows (samples), no of time steps (columns), no of features (close price)

In [45]:
x_test.shape

(201, 60, 1)

In [46]:
#Get the models predicted price (y-test)
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions) #get predicted y-test base of off x_test

In [47]:
#Get the root mean squared error (RMSE)
#Its just one of several ways to test model
# The smaller it is, the better, 0 means exactly 
rmse = np.sqrt(np.mean( predictions - y_test )**2 )
rmse

0.7410855269550685